# Imports

In [ ]:
!python -m spacy download pt_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-10 22:58:54.056448: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 3.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import pandas as pd
import numpy as np
import json
import pickle as pkl

import yaml

import uuid

import re
import spacy
from spacy.lang.pt.examples import sentences 
from spacy.training import offsets_to_biluo_tags

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Utils

In [ ]:
nlp = spacy.load('pt_core_news_sm')

entidadesExtratoContrato = ['data_assinatura_contrato',
                            'partes_contratuais',
                            'natureza_despesa',
                            'nota_empenho',
                            'numero_contrato',
                            'objeto_contrato',
                            'processo_gdf',
                            'programa_trabalho',
                            'unidade_orcamentaria',
                            'valor_contrato',
                            'vigencia_contrato'
                            ]

def get_token_for_char(doc, char_idx):
    for i, token in enumerate(doc):
        if  token.idx <= char_idx <= token.idx+len(token.text):
            return token.idx,len(token.text)

def get_span(text, label, ent):
    span = {'label':label, 'start':0, 'end':0}
    doc = nlp(text)
    if ent in doc.text:
        t1 = get_token_for_char(doc,doc.text.index(ent))
        t2 = get_token_for_char(doc,doc.text.index(ent)+len(ent))
        span['start'] = t1[0]
        span['end'] = t2[0]+t2[1]
        #print(doc.text[span['start']:span['end']])
    return span

def entity_to_span(text, value, key):
    span_list = []
    e=0
    text = re.sub('\n', ' ', text)
    for entity in value:
        doc = nlp(text)
        ent = re.sub('\n', ' ', entity.replace(r'\n', '\n'))
        #print(text, '\n\t\t',key,'->', ent,'\n\n')
        #print(ent)
        span = {'label':key, 'start':0, 'end':0}
        #ent = entity[0]
        if ent in doc.text:
            #print(ent)
            #print(doc.text)
            t1 = get_token_for_char(doc,doc.text.index(ent))
            #print(doc.text.index(ent)+len(ent)-1)
            t2 = get_token_for_char(doc,doc.text.index(ent)+len(ent))
            span['start'] = t1[0]
            span['end'] = t2[0]+t2[1]
            #print(doc.text[span['start']:span['end']])
            if(span['start']!=0 and span['end']!=0):
                span_list.append((span['start']+e,span['end']+e,span['label']))
            e+=span['end']
            text = text[span['end']:]
    return span_list

# testando prediçoes coflitantes
# '(948, 958, 'data_assinatura_contrato')' and '(907, 1101, 'vigencia_contrato')'
def ajust_preds(text,preds,preds2):
    #print(preds)
    text = re.sub('\n', ' ', text)
    for p1 in preds:
        for p2 in preds:
            #print(p1,p2)
            if ((p2[0]<=p1[0] and p1[0]<=p2[1]) or (p2[0]<=p1[1] and p1[1]<=p2[1])) and p1 != p2:
                # print('p1:', p1)
                # print('p2:', p2)
                ent = text[p1[0]:p1[1]]
                label = p1[2]
                #print(ent)
                text_aux = text[p2[1]:]
                idx0 = p2[0]
                idx1 = p2[1]
                flag = 0
                # Procura no texto por outra ocorrencia
                while flag == 0:
                    flag = 1
                    span = get_span(text_aux,label,ent)
                    #print(span)
                    for p3 in preds:
                        # testa se esta occorencia ja está em alguma outra entidade, se sim continua procurando, se não encontrou a ocorrencia desejada
                        if ((p3[0]<=(span['start']+idx1)<=p3[1]) or (p3[0]<=(span['end']+idx1)<=p3[1])) and (span['start']!=0 and span['end']!=0):
                            #print('p3[0]:',p3[0])
                            #print('p3[1]:',p3[1])
                            flag = 0
                            text_aux = text[p3[1]:]
                            #print(text_aux)
                            idx0 = p3[0]
                            idx1 = p3[1]
                            # print(f"{span['start']} + {idx1} = {span['start']+idx1}")
                            # print(f"{span['end']} + {idx1} = {span['end']+idx1}")
                            #break
                # se achou um acorrencia que não faz parte de outra entidade, remove sua ocorrencia anterior e adiciona a nova
                if(span['start']!=0 and span['end']!=0):
                    #print('trocou p1: ', p1)
                    #print('por: ', (span['start']+idx1,span['end']+idx1,span['label']))
                    preds.remove(p1)
                    preds.append((span['start']+idx1,span['end']+idx1,span['label']))
                    return preds,preds2,0
                # se não achou nenhuma nova ocorrencia, devemos retirar a ocorrencia de maior tamanho, pois é provavel que essa esteja afetando a ocorrencia menor
                else:
                    ent = text[p2[0]:p2[1]]
                    label = p2[2]
                    #print(ent)
                    text_aux = text[p1[1]:]
                    idx0 = p1[0]
                    idx1 = p1[1]
                    flag = 0
                    while flag == 0:
                        flag = 1
                        span = get_span(text_aux,label,ent)
                        #print(span)
                        for p3 in preds:
                            # testa se esta occorencia ja está em alguma outra entidade, se sim continua procurando, se não encontrou a ocorrencia desejada
                            if ((p3[0]<=(span['start']+idx1)<=p3[1]) or (p3[0]<=(span['end']+idx1)<=p3[1])) and (span['start']!=0 and span['end']!=0):
                                #print('p3[0]:',p3[0])
                                #print('p3[1]:',p3[1])
                                flag = 0
                                text_aux = text[p3[1]:]
                                #print(text_aux)
                                idx0 = p3[0]
                                idx1 = p3[1]
                                # print(f"{span['start']} + {idx1} = {span['start']+idx1}")
                                # print(f"{span['end']} + {idx1} = {span['end']+idx1}")
                                #break
                    if(span['start']!=0 and span['end']!=0):
                        #print('trocou p1: ', p1)
                        #print('por: ', (span['start']+idx1,span['end']+idx1,span['label']))
                        preds.remove(p2)
                        preds.append((span['start']+idx1,span['end']+idx1,span['label']))
                        return preds,preds2,0
                    else:
                        #print('removeu p1: ', p1)
                        preds2.append(p1)
                        preds.remove(p1)
                        return preds,preds2,0
    return preds,preds2,1

def mix_final_preds(final_preds):
    for i in range(len(final_preds)):
        for j in range(i,len(final_preds)):
            for p1 in final_preds[i]:
                flag = 1
                for p2 in final_preds[j]:
                    if ((p2[0]<=p1[0]<=p2[1]) or (p2[0]<=p1[1]<=p2[1]) or (p1[0]<=p2[0]<=p1[1]) or (p1[0]<=p2[1]<=p1[1])):
                        flag = 0
                if flag==1:
                    final_preds[j].append(p1)
    return final_preds

def rec_ajust_preds(text,preds):
    final_preds = []
    preds2 = []
    teste = 0
    while teste != 1:
        preds,preds2, teste = ajust_preds(text,preds,preds2)
    if(preds2==[]):
        return [preds]
    else:
        final_preds.append(preds)
        for pred in rec_ajust_preds(text,preds2):
            final_preds.append(pred)
        # Junta as predições que não sobresaeem uma da outra
        return mix_final_preds(final_preds)

def get_list_spans(text,labels_dict):
    #labels_dict = yaml.safe_load(labels_dict)
    lista = []
    for key, value in labels_dict.items():
        if key not in entidadesExtratoContrato:
          continue
        #pred = ajust_preds(text,entity_to_span(text,value,key))
        pred = entity_to_span(text,value,key)
        for i in pred:
            lista.append(i)
    #print(lista)
    return rec_ajust_preds(text,lista)

In [ ]:
def tokenize(texto,labels):
    if isinstance(labels, str):
        labels = yaml.safe_load(re.sub('\n', ' ', labels.replace(r'\n', '\n')))
        
    nlp = spacy.load('pt_core_news_sm')
    preds = get_list_spans(texto,labels)

    conll=''
    for pred in preds:

        text = re.sub('\n', ' ', texto)
        doc = nlp(text)

        tags = offsets_to_biluo_tags(doc, pred)

        for i in range(len(doc)):
            if tags[i][0] == 'U':
                conll += doc[i].text + ' ' + 'B' + tags[i][1:] + '\n'
            elif tags[i][0] == 'L':
                conll += doc[i].text + ' ' + 'I' + tags[i][1:] + '\n'
            else:
                conll += doc[i].text + ' ' + tags[i] + '\n'
              
        conll+='\n'

    return conll

In [ ]:
# Função que gera string no formato Conll a partir das predições dos modelos
def get_conll_conc(pred):
  conll_conc = ""
  for i, tupla in enumerate(pred):
      txt,ent = tupla[0],tupla[1]
      if ent =='O':
        conll_conc += txt + ' O\n'
      else:
        if(i!=0 and pred[i-1][1]==ent):
          conll_conc += txt + ' '+ ent +'\n'
        else:
          conll_conc += txt + ' '+ ent +'\n'

  return conll_conc+'\n'

In [ ]:
# função que trasnforma as prediçõe dos modelos ktrain em dicionarios para facilitar a comparação sem depender de tokenizações
def get_entities_from_prediction(predictions):
    lista = []
    cont = 0
    for pred in predictions:
        ents_dit = {}
        i=0
        while (i < len(pred)):
            txt,ent = pred[i][0],pred[i][1]
            if ent !='O':
                aux_txt = txt
                while(len(pred) > i+1 and pred[i+1][1] != 'O'):
                    if(pred[i+1][1][2:] == ent[2:]):
                        aux_txt += ' ' + pred[i+1][0]
                        i+=1
                    else:
                        break
                if(ent[2:] in ents_dit):
                    ents_dit[ent[2:]].append(aux_txt)
                else:
                    ents_dit[ent[2:]] = [aux_txt]
            i+=1
        lista.append(ents_dit)
        cont+=1
    return lista

# Carregando dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import gdown

# url = 'https://drive.google.com/drive/folders/1AJitYBCAc-vI5kgcAGEVV5oD4q9EmVoZ?usp=sharing'

# gdown.download_folder(url)

In [ ]:
df_base = pd.read_csv('/content/drive/MyDrive/tccF/df_base.csv')
df_base.drop('Unnamed: 0', axis=1, inplace=True)
df_base

,id,id_ato,texto,label,pred
0,0,12_22.11.2018-R2,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'partes_contratuais': ['SEAGRI/DF', 'CIMAG C...","{'natureza_despesa': ['449052'], 'partes_contr..."
1,1,12_22.11.2018-R3,EXTRATO DO CONTRATO No 32/2018 AQUISICAO DE BE...,"{'numero_contrato': ['32/2018'], 'processo_gdf...",{'partes_contratuais': ['INSTITUTO DE ASSIST. ...
2,2,12_22.11.2018-R7,"EXTRATO DO CONTRATO No 01/2018, NOS TERMOS DO ...","{'numero_contrato': ['01/2018'], 'processo_gdf...","{'natureza_despesa': ['449051'], 'partes_contr..."
3,3,12_22.11.2018-R5,EXTRATO DO CONTRATO PARA PRESTACAO DE SERVICOS...,"{'numero_contrato': ['07/2018'], 'processo_gdf...","{'natureza_despesa': ['3.3.90.39.11'], 'partes..."
4,4,12_22.11.2018-R6,EXTRATO DO CONTRATO TEC2B SOLUTIONS No 15/2018...,"{'partes_contratuais': ['IBRAM/DF', 'TEC2B SOL...","{'partes_contratuais': ['IBRAM/DF'], 'objeto_c..."
...,...,...,...,...,...
778,778,638-R75,EXTRATO DO CONTRATO BRB Nº 140/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Prosegur Brasi...","{'natureza_despesa': ['4'], 'partes_contratuai..."
779,779,638-R76,EXTRATO DO CONTRATO BRB Nº 141/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Renaforte Serv...",{'partes_contratuais': ['BRASILIA RESERVADO AO...
780,780,638-R78,EXTRATO DO CONTRATO BRB Nº 159/2022 Contratado...,"{'partes_contratuais': ['BRB', 'Russell Bedfor...","{'natureza_despesa': ['4'], 'partes_contratuai..."
781,781,638-R81,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...,"{'data_assinatura_contrato': ['15/03/2022'], '...","{'partes_contratuais': ['SEEDF', 'SEEDF'], 'ob..."


#SPLITANDO

In [ ]:
data = df_base.sample(frac=0.1, random_state=25)

df_10 = data
df_10.reset_index(inplace=True,drop=True)
print(len(df_10))

78


In [ ]:
data = df_base.sample(frac=0.2, random_state=25)

df_20 = data
df_20.reset_index(inplace=True,drop=True)
print(len(df_20))

157


In [ ]:
data = df_base.sample(frac=0.3, random_state=25)

df_30 = data
df_30.reset_index(inplace=True,drop=True)
print(len(df_30))

235


In [ ]:
data = df_base.sample(frac=0.4, random_state=25)

df_40 = data
df_40.reset_index(inplace=True,drop=True)
print(len(df_40))

313


In [ ]:
data = df_base.sample(frac=0.5, random_state=25)

df_50 = data
df_50.reset_index(inplace=True,drop=True)
print(len(df_50))

392


In [ ]:
data = df_base.sample(frac=0.6, random_state=25)

df_60 = data
df_60.reset_index(inplace=True,drop=True)
print(len(df_60))

470


In [ ]:
data = df_base.sample(frac=0.7, random_state=25)

df_70 = data
df_70.reset_index(inplace=True,drop=True)
print(len(df_70))

548


In [ ]:
data = df_base.sample(frac=0.8, random_state=25)

df_80 = data
df_80.reset_index(inplace=True,drop=True)
print(len(df_80))

626


In [ ]:
data = df_base.sample(frac=0.9, random_state=25)

df_90 = data
df_90.reset_index(inplace=True,drop=True)
print(len(df_90))

705


# Gerando CONLLs

In [ ]:
conll_10 = ""
for index,row in df_10.iterrows():
  conll_10 += tokenize(row['texto'],row['label'])

In [ ]:
print(conll_10[0:20000])

EXTRATO O
DO O
CONTRATO O
DE O
CREDENCIAMENTO O
Nº O
91/2021 B-numero_contrato
PRESTAÇÃO O
DE O
SERVIÇOS O
  O
Processo O
: O
00053-00121023/ B-processo_gdf
2020-48 I-processo_gdf
. O
Partes O
: O
CBMDF B-partes_contratuais
X O
RECRIAR B-partes_contratuais
CLÍNICA I-partes_contratuais
PSICOPEDAGÓGICA I-partes_contratuais
LTDA I-partes_contratuais
, O
CNPJ O
nº O
20.236.933/ O
0001-90 O
. O
Objeto O
: O
a B-objeto_contrato
prestação I-objeto_contrato
de I-objeto_contrato
serviços I-objeto_contrato
especializados I-objeto_contrato
em I-objeto_contrato
serviços I-objeto_contrato
de I-objeto_contrato
psicologia I-objeto_contrato
e I-objeto_contrato
psiquiatria I-objeto_contrato
, I-objeto_contrato
fisioterapia I-objeto_contrato
, I-objeto_contrato
fonoaudiologia I-objeto_contrato
, I-objeto_contrato
pediatria I-objeto_contrato
, I-objeto_contrato
terapia I-objeto_contrato
ocupacional I-objeto_contrato
e I-objeto_contrato
nutrição I-objeto_contrato
, I-objeto_contrato
conforme I-objeto_cont

In [ ]:
conll_20 = ""
for index,row in df_20.iterrows():
  conll_20 += tokenize(row['texto'],row['label'])

In [ ]:
conll_30 = ""
for index,row in df_30.iterrows():
  conll_30 += tokenize(row['texto'],row['label'])

In [ ]:
conll_40 = ""
for index,row in df_40.iterrows():
  conll_40 += tokenize(row['texto'],row['label'])

In [ ]:
conll_50 = ""
for index,row in df_50.iterrows():
  conll_50 += tokenize(row['texto'],row['label'])

In [ ]:
conll_60 = ""
for index,row in df_60.iterrows():
  conll_60 += tokenize(row['texto'],row['label'])

In [ ]:
conll_70 = ""
for index,row in df_70.iterrows():
  conll_70 += tokenize(row['texto'],row['label'])

In [ ]:
conll_80 = ""
for index,row in df_80.iterrows():
  conll_80 += tokenize(row['texto'],row['label'])

In [ ]:
conll_90 = ""
for index,row in df_90.iterrows():
  conll_90 += tokenize(row['texto'],row['label'])

## salvando conlls

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Basic/conll_10.conll', 'w') as f:
    f.write(conll_10)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Basic/conll_20.conll', 'w') as f:
    f.write(conll_20)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Basic/conll_20.conll', 'w') as f:
    f.write(conll_20)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Basic/conll_30.conll', 'w') as f:
    f.write(conll_30)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Basic/conll_40.conll', 'w') as f:
    f.write(conll_40)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Basic/conll_50.conll', 'w') as f:
    f.write(conll_50)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Basic/conll_60.conll', 'w') as f:
    f.write(conll_60)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Basic/conll_70.conll', 'w') as f:
    f.write(conll_70)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Basic/conll_80.conll', 'w') as f:
    f.write(conll_80)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Basic/conll_90.conll', 'w') as f:
    f.write(conll_90)

# Concatenando conll

In [ ]:
with open('/content/drive/MyDrive/tccF/base_conll_GPT.conll', 'r') as f:
    base_conll_GPT = f.read()

with open('/content/drive/MyDrive/tccF/base_conll_human.conll', 'r') as f:
    base_conll_human = f.read()

In [ ]:
print(base_conll_GPT[0:10000])

EXTRATO O
DO O
CONTRATO O
PARA O
AQUISICAO O
DE O
BENS O
No O
47/2018 B-numero_contrato
PROCESSO O
: O
00070-00016822/ B-processo_gdf
2018-99 I-processo_gdf
. O
  O
Partes O
: O
SEAGRI B-partes_contratuais
/ I-partes_contratuais
DF I-partes_contratuais
  O
e O
CIMAG O
  O
COMERCIO O
DE O
IMPLEMENTOS O
E O
MAQUINAS O
AGRICOLAS O
LTDA O
. O
Objeto O
: O
O B-objeto_contrato
Contrato I-objeto_contrato
tem I-objeto_contrato
por I-objeto_contrato
objeto I-objeto_contrato
a I-objeto_contrato
aquisicao I-objeto_contrato
dos I-objeto_contrato
seguintes I-objeto_contrato
itens I-objeto_contrato
: I-objeto_contrato
Item I-objeto_contrato
20 I-objeto_contrato
: I-objeto_contrato
Quantidade I-objeto_contrato
05 I-objeto_contrato
( I-objeto_contrato
cinco I-objeto_contrato
) I-objeto_contrato
. I-objeto_contrato
GRADE I-objeto_contrato
ARADORA I-objeto_contrato
CONTROLE I-objeto_contrato
REMOTO I-objeto_contrato
, I-objeto_contrato
com I-objeto_contrato
14 I-objeto_contrato
discos I-objeto_contrato


In [ ]:
print(base_conll_human[0:6000000])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
conll_110 = base_conll_GPT + conll_10
conll_120 = base_conll_GPT + conll_20
conll_130 = base_conll_GPT + conll_30
conll_140 = base_conll_GPT + conll_40
conll_150 = base_conll_GPT + conll_50
conll_160 = base_conll_GPT + conll_60
conll_170 = base_conll_GPT + conll_70
conll_180 = base_conll_GPT + conll_80
conll_190 = base_conll_GPT + conll_90
conll_200 = base_conll_GPT + base_conll_human

## salvando conlls

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Combined/conll_110.conll', 'w') as f:
    f.write(conll_110)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Combined/conll_120.conll', 'w') as f:
    f.write(conll_120)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Combined/conll_130.conll', 'w') as f:
    f.write(conll_130)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Combined/conll_140.conll', 'w') as f:
    f.write(conll_140)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Combined/conll_150.conll', 'w') as f:
    f.write(conll_150)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Combined/conll_160.conll', 'w') as f:
    f.write(conll_160)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Combined/conll_170.conll', 'w') as f:
    f.write(conll_170)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Combined/conll_180.conll', 'w') as f:
    f.write(conll_180)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Combined/conll_190.conll', 'w') as f:
    f.write(conll_190)

In [ ]:
with open('/content/drive/MyDrive/Conll_IT/Combined/conll_200.conll', 'w') as f:
    f.write(conll_200)

In [ ]:
with open('/content/drive/MyDrive/tccF/base_conll_GPT.conll', 'r') as f:
    tes_len = f.read()

tes_len=len(tes_len.split("\n\n"))
print(tes_len)

1064


In [ ]:
with open('/content/drive/MyDrive/tccF/base_conll_human.conll', 'r') as f:
    tes_len = f.read()

tes_len=len(tes_len.split("\n\n"))
print(tes_len)


907
